You can download all data from [All data](https://drive.google.com/open?id=1Xxp055WzGNsF_J8lxR2_5pehlnb0Igco). The following is my code

In [1]:
from google.colab import drive
drive.mount('/drive',force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


In [2]:
import json
import nltk
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import gensim.corpora as corpora
from matplotlib import pyplot as plt
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import string
import numpy as np
from numpy import random
import pandas as pd
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

#0. Load Data

In [4]:
import os
directory = os.listdir('./wiki-pages/wiki-pages/')
print(len(directory))
file_list=[]
for file in directory:
  file_list.append(file)

109


##Remove stopword word & punctuation

In [0]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  

stop_words = stopwords.words('english')

def get_tokens(text):
    lower = text.lower()
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lower.translate(remove_punctuation_map)
    tokens = nltk.word_tokenize(no_punctuation)
    tokens=[word for word in tokens if word not in ['rrb','lrb']]
    return [word for word in tokens if word not in stop_words]
  
def lemmatized_tokens(tokens, lemmatizer):
    lemmatized = []
    for item in tokens:
        lemmatized.append(lemmatizer.lemmatize(item))
    return lemmatized

##Save and Load data to dictory

In [0]:
import numpy as np

def save_dict(filename,dic):
  np.save(filename, dic) 

def load_dict(filename):
  read_dictionary = np.load(filename,allow_pickle=True).item()
  return read_dictionary

#1. Task1: Text Statistics

In [0]:
def traverse_directory_all(list_dir):
  dict1={}
  count=1
  for file in list_dir:
      print("file name is "+ str(file))
      test=open('/drive/My Drive/IRDM/wiki-pages/wiki-pages/'+file)  
      for line in test:
        json_obj=json.loads(line)
        line=get_tokens(json_obj['text'])
        for term in line:
          if term in dict1:
            dict1[term]=dict1[term]+1
          else:
            dict1[term]=1
      print(count)
      count+=1
      test.close()
  save_dict('/drive/My Drive/IRDM/task1/task1_dict.npy',dict1)
  return dict1

In [0]:
traverse_directory_all(file_list)

In [0]:
dict1=load_dict('/drive/My Drive/IRDM/task1/task1_dict.npy')
ranked=sorted(dict1.items(),key=lambda item:item[1],reverse=True)
x=[]
y=[]

total=np.int64(0)
total=total+np.int64(sum(dict1.values()))
statis=[]

for i in range(len(ranked)):
  x.append(i+1)
  y.append(ranked[i][1])
  
for i in range(30):
  statis.append([ranked[i][0],ranked[i][1],i+1,float(ranked[i][1]/total),(i+1)*float(ranked[i][1]/total)])

print(statis)
plt.plot(x,y)
plt.xlabel('ranked')
plt.ylabel('frequencey')
plt.xscale('log')
plt.yscale('log')
plt.savefig('/drive/My Drive/IRDM/zip.jpg')

# 2. Task2: TF-IDF


##TF-IDF

In [0]:
from collections import Counter
for file in file_list:
    file=file.split(".")[0]
    print(file)
    test_file=open("/drive/My Drive/IRDM/wiki-pages/wiki-pages/"+str(file)+".jsonl")
    test_output=open("/drive/My Drive/IRDM/task2/frequency/"+str(file)+".txt",'w')
    output=open("/drive/My Drive/IRDM/task2/tokens/"+str(file)+".txt",'w')
    for line in test_file:
      json_obj=json.loads(line)
      tokens=get_tokens(json_obj['text'])
      res_id=json_obj['id']
      lemmatizer=WordNetLemmatizer()  
      res_tokens=lemmatized_tokens(tokens, lemmatizer)
      count = Counter(res_tokens)
      temp=count.items()
      test_output.writelines("id:"+str(res_id)+"#tokens:")
      for key,value in temp:
        test_output.writelines(str(key)+":"+str(value)+',')
      test_output.writelines("\n")
      output.writelines("id:"+str(res_id)+";tokens:"+str(res_tokens)+'\n')

    test_output.close()
    output.close()
  

In [0]:
def train_count():
  train=open("/drive/My Drive/IRDM/train_temp.jsonl")
  output=open("/drive/My Drive/IRDM/task2/train.txt",'w')
  for line in train:
    json_obj=json.loads(line)
    tokens=get_tokens(json_obj['claim'])
    res_id=json_obj['id']
    lemmatizer=WordNetLemmatizer()  
    res_tokens=lemmatized_tokens(tokens, lemmatizer)
    count = Counter(res_tokens)
    temp=count.items()
    output.writelines("id:"+str(res_id)+"#tokens:")
    for key,value in temp:
      output.writelines(str(key)+":"+str(value)+',')
    output.writelines("\n")
    
  output.close()
  

In [0]:
import json
import os
import math

directory = os.listdir('/drive/My Drive/IRDM/task2/frequency')


def tf_claim():
    claim=open('/drive/My Drive/IRDM/task2/train.txt','r')
    for i in claim.readlines():
        dict_temp={}
        id,tokens=i.strip().split("#")
        print(id)
        tokens=tokens[7:len(tokens)-1]
        token_list=tokens.split(",")
        for i1 in token_list:
            word,fre=i1.split(":")
            dict_temp[word]=int(fre)

        for i2 in dict_temp.keys():
            tf=dict_temp[i2]/sum(dict_temp.values())
            print(i2+":"+str(tf))

def claim_text():
    claim=open('/drive/My Drive/IRDM/task2/train.txt','r')
    dict={}
    for i in claim.readlines():
        text_list=[]
        id,tokens=i.strip().split("#")
        id=id.split(":")[1]
        tokens=tokens[7:len(tokens)-1]
        token_list=tokens.split(",")
        for i1 in token_list:
            word,fre=i1.split(":")
            text_list.append(word)
        dict[str(id)]=text_list

    return dict

def tf_idf_doc(claim_id,text):
    dict_text_num={}
    dict_id_tf={}
    dict_text_idf={}
    for i2 in text:
        dict_text_num[i2] = 0

    for file_name in file_list:
        print(file_name)
        file=open("/drive/My Drive/IRDM/task2/frequency/"+file_name,'r')
        for i in file.readlines():
            dict_temp={}
            id,tokens=i.strip().split("#")
            id=id.split(":")[1]
            if(tokens!='tokens:'):
                tokens=tokens[7:len(tokens)-1]
                token_list=tokens.split(",")
                for i1 in token_list:
                    word,fre=i1.split(":")
                    dict_temp[word]=int(fre)
            tf_list = []
            for i2 in text:
                if(i2 in dict_temp):
                    #print(i2)
                    dict_text_num[i2]+=1
                    tf=dict_temp[i2]/sum(dict_temp.values())
                    tf_list.append(float(tf))
                    #print(tf)
                else:
                    tf=0
                    tf_list.append(float(tf))
            dict_id_tf[id]=tf_list

    output1=open("/drive/My Drive/IRDM/task2/"+str(claim_id)+"_id_tf.txt",'w')
    dict_id_tf_new={}
    for text3,number3 in dict_id_tf.items():
        for i4 in number3:
            if(i4!=0):
                output1.writelines(text3+":"+str(number3)+"\n")
                dict_id_tf_new[text3]=number3
                break
    output1.close()

    output2=open("/drive/My Drive/IRDM/task2/"+str(claim_id)+"_idf.txt",'w')
    for text1,number in dict_text_num.items():
        idf=math.log(5395867)/(1+number)
        dict_text_idf[text1]=float(idf)
        output2.writelines(text1+":"+str(idf)+'\n')

    idf_vector = []
    for text2,docu_idf in dict_text_idf.items():
        idf_vector.append(docu_idf)

    dict_id_tf_idf={}
    for text4,tf in dict_id_tf_new.items():
        tf_idf_vector=[]
        for i5 in range(len(idf_vector)):
            tf_idf=float(tf[i5])*float(idf_vector[i5])
            tf_idf_vector.append(float(tf_idf))
        dict_id_tf_idf[text4]=tf_idf_vector

    output3=open("/drive/My Drive/IRDM/task2/"+str(claim_id)+"_tf_idf.txt",'w')
    for text5,tf_idf_now in dict_id_tf_idf.items():
        output3.writelines("id:"+text5+"#"+"tf-idf:"+str(tf_idf_now)+'\n')


In [0]:
text=claim_text()
for key,value in text.items():
    tf_idf_doc(key,value)

##Cosine Similarity

In [0]:
import pandas as pd
import numpy as np

def top_five_cos_sim():
    claim_tf_idf=open("/drive/My Drive/IRDM/task2/claim-tf-idf/claim_tf-idf.txt",'r')
    claim_tf_idf_dict={}
    for i in claim_tf_idf.readlines():
        id_1,vec=i.strip().split(";")
        claim_id=id_1.split(":")[1]
        vec=vec[1:len(vec)-1].split(",")
        tf_idf_vec=[]
        for i1 in vec:
            tf_idf_vec.append(float(i1))
        claim_tf_idf_dict[claim_id]=tf_idf_vec

    directory = os.listdir('/drive/My Drive/IRDM/doc-tf-idf/')
    file_list = []
    for file in directory:
        file_list.append(file)

    for i2 in file_list:
        id_claim=i2.split("_",1)[0]
        print(id_claim)
        file_temp=open("/drive/My Drive/IRDM/doc-tf-idf/"+i2,'r')
        claim_tf_idf_vec=pd.Series(claim_tf_idf_dict[id_claim])
        cos_dict={}
        for i3 in file_temp.readlines():
            id_2,vec=i3.split("#")
            id_doc=id_2.split(":",1)[1]
            vec=vec.split(':',1)[1]
            vec=vec[1:len(vec)-2]

            doc_tf_idf=[]
            for i4 in vec.split(","):
                doc_tf_idf.append(float(i4))
            doc_tf_id_vec=pd.Series(doc_tf_idf)
            cos_sim_value=cos_sim(doc_tf_id_vec,claim_tf_idf_vec)
            cos_dict[id_doc]=cos_sim_value

        top_five=order_dict_N(cos_dict,5)
        output=open('/drive/My Drive/IRDM/'+id_claim+"_cos_sim_value.txt",'w')
        for i5 in top_five:
            output.writelines(str(i5)+"\n")

        output.close()
        print(id_claim+" finish!!")

def order_dict_N(dict, N):
    b = sorted([(k, v) for k, v in dict.items()], reverse=True)
    e = set()
    for i in b:
        e.add(i[1])
    result = []
    for i in sorted(e, reverse=True)[:N]:
        for j in b:
           if j[1] == i:
              result.append(j)
    return result


def cos_sim(vector_a, vector_b):
    """
    :param vector_a:  a
    :param vector_b:  b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim


In [0]:
top_five_cos_sim()

#3. Task3: Probabilistic Document Retrieval

In [0]:
import pandas as pd
def traverse_file_each(list_dir):
  for file in list_dir:
    print("file name is :" +str(file))
    csv=pd.DataFrame()
    test=open('/drive/My Drive/IRDM/wiki-pages/wiki-pages/'+file)  
    for line in test:
      fre_dict={}
      lines_arr=[]
      json_obj=json.loads(line)
      lines=json_obj['lines'].split('\n')
      for i in range(len(lines)):
        temp=lines[i][2:]
        if len(temp)>1:
          lines_arr.append(temp
      tokens=get_tokens(json_obj['text'])
      lemmatizer=WordNetLemmatizer()
      line=lemmatized_tokens(tokens, lemmatizer)
      for term in line:
        if term in fre_dict:
          fre_dict[term]=fre_dict[term]+1
        else:
          fre_dict[term]=1
      csv=csv.append({'id':json_obj['id'],'freq':fre_dict,'lines':lines_arr},ignore_index=True)
    csv.to_csv('/drive/My Drive/IRDM/task3/document_new/'+file+'.csv')
    test.close()

In [0]:
traverse_file_each(file_list)

##Query-likelyhood

In [0]:
import pandas as pd

def query_likelyhood()
  all_docu = os.listdir('/drive/My Drive/IRDM/task3/document/')
  document_result={}
  claim=open('/drive/My Drive/IRDM/train_temp.jsonl')
  claim_dict={}

  for line in claim:
    json_obj=json.loads(line)
    line=get_tokens(json_obj['claim'])
    claim_dict[json_obj['id']]=line
    document_result[json_obj['id']]={}

  for document in all_docu:
    print(document)
    all_freq={}
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document)
    temp_freq=temp_csv['freq'].values
    temp_id=temp_csv['id'].values
    for i in range(len(temp_freq)):
      all_freq[temp_id[i]]=eval(temp_freq[i])
    for one_claim in claim_dict.keys():
        for document in all_freq.keys():
          document_result[one_claim][document]=1
          temp_total=np.int64(np.sum(list(all_freq[document].values())))
          for word in claim_dict[one_claim]:
            if word in all_freq[document].keys():
              document_result[one_claim][document]= document_result[one_claim][document]*all_freq[document][word]/temp_total
            else:
              document_result[one_claim][document]=0
              break
  save_dict('/drive/My Drive/IRDM/task3/task3_query_dict.npy',document_result)


In [0]:
query_likelyhood()

##Laplace smoothing

In [0]:
import pandas as pd

def laplace_smoothing(file_list):
  vocab_size=len(load_dict('/drive/My Drive/IRDM/task1/task1_dict_1.npy').keys())
  for document in file_list:
    document_result={}
    claim=open('/drive/My Drive/IRDM/train_temp.jsonl')
    claim_dict={}
    for line in claim:
      json_obj=json.loads(line)
      line=get_tokens(json_obj['claim'])
      claim_dict[json_obj['id']]=line
      document_result[json_obj['id']]={}  
    print(document)
    document_score={}
    all_freq={}
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document)
    temp_freq=temp_csv['freq'].values
    temp_id=temp_csv['id'].values
    for i in range(len(temp_freq)):
      all_freq[temp_id[i]]=eval(temp_freq[i])
      document_score[temp_id[i]]=1
    for single_claim in claim_dict.keys():
        for doc in all_freq.keys():
          document_result[single_claim][doc]=1
          temp_total=np.int64(np.sum(list(all_freq[doc].values())))

          if(temp_total+vocab_size)==0:
            document_result[single_claim][doc]=0
            continue
          for word in claim_dict[single_claim]:

            if word in all_freq[doc].keys():
              document_result[single_claim][doc]= document_result[single_claim][doc]*(all_freq[doc][word]+1)/(temp_total+vocab_size)
            else:
              document_result[single_claim][doc]= document_result[single_claim][doc]*(0+1)/(temp_total+vocab_size)
    save_dict('/drive/My Drive/IRDM/task3/laplace/task3_laplace_dict_'+document+'.npy',document_result)
  

In [0]:
file_list = os.listdir('/drive/My Drive/IRDM/task3/document/')
laplace_smoothing(file_list)

##Jelinek-Mercer Smoothing

In [0]:
def jel_smoothing(file_list):
  vocab_size=len(load_dict('/drive/My Drive/IRDM/task1/task1_dict.npy').keys())
  for document in file_list:
    print(document)
    document_score={}
    all_freq={}
    document_result={}
    claim=open('/drive/My Drive/IRDM/train_temp.jsonl','r')
    claim_dict={}
    for line in claim:
      json_obj=json.loads(line)
      line=get_tokens(json_obj['claim'])
      claim_dict[json_obj['id']]=line
      document_result[json_obj['id']]={}
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document,chunksize=30000)
    for chunk in temp_csv:
      temp_freq=chunk['freq'].values
      temp_id=chunk['id'].values
      for i in range(len(temp_freq)):
        all_freq[temp_id[i]]=eval(temp_freq[i])
        document_score[temp_id[i]]=1
    all_freq[0]=load_dict('/drive/My Drive/IRDM/task1/task1_dict.npy')
  
  
    lamb=1/2
    V=np.int64(np.sum(list(all_freq[0].values())))
    
    for one_claim in claim_dict.keys():
        for doc in all_freq.keys():
          if(doc==0):
            continue
          document_result[one_claim][doc]=1
          temp_total=np.int64(np.sum(list(all_freq[doc].values())))

          for word in claim_dict[one_claim]:

            if word in all_freq[doc].keys():
              if word in all_freq[0]:
                document_result[one_claim][doc]= document_result[one_claim][doc]*((1-lamb)*(all_freq[doc][word]+1)/(temp_total+vocab_size)+lamb*all_freq[0][word]/V)
              else:
                document_result[one_claim][doc]= document_result[one_claim][doc]*((1-lamb)*(all_freq[doc][word]+1)/(temp_total+vocab_size))
            else:
              if word in all_freq[0]:
                document_result[one_claim][doc]= document_result[one_claim][doc]*((1-lamb)*(0+1)/(temp_total+vocab_size)+lamb*all_freq[0][word]/V)
              else:
                document_result[one_claim][doc]= document_result[one_claim][doc]*((1-lamb)*(0+1)/(temp_total+vocab_size))
    
    save_dict('/drive/My Drive/IRDM/task3/jel/task3_jel_dict_'+str(document)+'.npy',document_result) 

file_list = os.listdir('/drive/My Drive/IRDM/task3/document/')
jel_smoothing(file_list)

##dirichlet Smoothing

In [0]:
def dirichlet_smoothing(file_list):
  for document in documents_traverse:
    print(document)
    document_score={}
    all_freq={}
    document_result={}
    claim=open('/drive/My Drive/IRDM/train_temp.jsonl')
    claim_dict={}
    for line in claim:
      json_obj=json.loads(line)
      line=get_tokens(json_obj['claim'])
      claim_dict[json_obj['id']]=line
      document_result[json_obj['id']]={}
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document)
    
    for part in temp_csv:
      temp_freq=part['freq'].values
      temp_id=part['id'].values
      for i in range(len(temp_freq)):
        all_freq[temp_id[i]]=eval(temp_freq[i])
        document_score[temp_id[i]]=1
    all_freq[0]=load_dict('/drive/My Drive/IRDM/task1/task1_dict.npy')
    
    mu=np.int64(np.sum(list(all_freq[0].values())))/(len(documents_traverse)*50000)
    V=np.int64(np.sum(list(all_freq[0].values())))
    for one_claim in claim_dict.keys():
        count=0
        for doc in all_freq.keys():
          if(doc==0):
            continue
          document_result[one_claim][doc]=1
          temp_total=np.int64(np.sum(list(all_freq[doc].values())))
          N=temp_total
          lamb=1-N/(N+mu)
          for word in claim_dict[one_claim]:

            if word in all_freq[doc].keys():
              if word in all_freq[0]:
                document_result[one_claim][doc]= document_result[one_claim][doc]*((1-lamb)*(all_freq[doc][word])/temp_total+lamb*all_freq[0][word]/V)
              else:
                document_result[one_claim][doc]= document_result[one_claim][doc]*((1-lamb)*(all_freq[doc][word])/temp_total)
            else:
              if word in all_freq[0]:
                document_result[one_claim][doc]= document_result[one_claim][doc]*(lamb*all_freq[0][word]/V)
              else:
                document_result[one_claim][doc]= 0
                break
    
    save_dict('/drive/My Drive/IRDM/task3/dirichlet/task3_Dirichlet_dict_'+document+'.npy',document_result) 
    


In [0]:
file_list = os.listdir('/drive/My Drive/IRDM/task3/document/')    
dirichlet_smoothing(file_list)

##Smoothing Analyze

In [0]:
query_dict=load_dict('/drive/My Drive/IRDM/task3/task3_query_dict.npy')
for claim in query_dict.keys():
  query_dict[claim]=sorted(query_dict[claim].items(),key=lambda item:item[1],reverse=True)
  print(claim)
  print(query_dict[claim][:5])

In [0]:
"""function for task3 smoothing analysis"""

def find_five_max(path):
  docu_traverse = os.listdir(path)
  claim=dict.fromkeys([75397, 150448, 214861, 156709, 129629, 33078, 6744, 226034, 40190, 76253])
  for key in claim.keys():
      claim[key]=[]
  for docu in docu_traverse:
    temp_dict=load_dict(path+str(docu))
    for key in temp_dict.keys():
        ranked = sorted(temp_dict[key].items(), key=lambda item: item[1], reverse=True)
        claim[key]=claim[key]+ranked[:5]
    
  for key in claim.keys():
      claim[key]=sorted(claim[key],key=lambda item:item[1],reverse=True)
      print(key)
      for item in claim[key][:5]
        print(item+";")
      print(" ")
  

# Laplace smoothing
find_five_max('/drive/My Drive/IRDM/task3/laplace/')

#  Jelinek-MercerSmoothing 
find_five_max('/drive/My Drive/IRDM/task3/jel/')

#  Dirichlet Smoothing 
find_five_max('./drive/My Drive/IRDM/task3/dirichlet/')


#Task4

##Trainset generation

In [0]:
import pandas as pd

def task4_train_set_generate(documents):
  for document in documents:
    print(document)
    train_set=pd.DataFrame()
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document)
    train=open('/drive/My Drive/IRDM/train.jsonl','r')
    length=len(temp_csv)
    count=0
    for line in train:
      count+=1
      if(count%1000==0):
        print("current line: "+str(count))
      json_obj=json.loads(line)
      if json_obj['verifiable']!='VERIFIABLE':
        continue
      else:
        evidence=json_obj['evidence']
        #print(evidence)
        info={}
        for item in evidence:
          for sub_item in item:
            if sub_item[2] in info:
              info[sub_item[2]].append(sub_item[3])
            else:
              info[sub_item[2]]=[]
              info[sub_item[2]].append(sub_item[3])
              
        for id_1 in info.keys():
          sentence_line=temp_csv[temp_csv['id']==id_1]
          if len(sentence_line)!=1:
            continue
          else:
            sentence=''
            for line_num in info[id_1]:
              arr=eval(sentence_line['lines'].values[0])
              if(line_num>=len(arr)):
                continue
              sentence=sentence+arr[line_num]
            r_idx=random.randint(length)
            negative_line=eval(temp_csv.iloc[r_idx]['lines'])
            while len(negative_line)==0:
              r_idx=random.randint(length)
              negative_line=eval(temp_csv.iloc[r_idx]['lines'])
            neg_sentence=negative_line[random.randint(len(negative_line))]
            while neg_sentence==sentence:
              r_idx=random.randint(length)
              negative_line=eval(temp_csv.iloc[r_idx]['lines'])

            neg_sentence=negative_line[random.randint(len(negative_line))]
            train_set=train_set.append({'claim':json_obj['claim'],'sentence':sentence,'label':1},ignore_index=True)
            train_set=train_set.append({'claim':json_obj['claim'],'sentence':neg_sentence,'label':0},ignore_index=True)
    train_set.to_csv('/drive/My Drive/IRDM/task4/task4_trainset/'+document[:8]+'_train.csv')



In [0]:
def generate_train_set(bound1,bound2)
    docu_traverse = os.listdir('/drive/My Drive/IRDM/task3/document/')
    docu_traverse=sorted(docu_traverse)
    bound=[bound1,bound2]
    documents=documents_traverse[bound[0]:bound[1]]
    task4_train_set_generate(documents)

In [0]:
import pandas as pd
import os

def merge_file(path_input,output_file):
  output = r'/drive/My Drive/IRDM/'+output_file
  file_list = os.listdir('/drive/My Drive/IRDM/'+path_input) 
  df = pd.read_csv('/drive/My Drive/IRDM/'+path_input+"/"+file_list[0])
  df=df.drop_duplicates(['claim','label','sentence'])
  df.to_csv(output, encoding="utf_8", index=False) 
  for i in range(1, len(file_list)):
      df = pd.read_csv('/drive/My Drive/IRDM/'+path'/'+file_list[i])
      df=df.drop_duplicates(['claim','label','sentence'])
      df.to_csv(output,encoding='utf-8' index=False, header=False, mode='a+')

merge_file('/task4/','tas4_train.csv')

##Word embedding

In [0]:
def word_averaging(wv, words):
    mean = []
    contain_wv_word = False
    for word in words:
        if word in wv.vocab:
            contain_wv_word = True
            mean.append(wv.vectors_norm[wv.vocab[word].index])
    if not contain_wv_word: #unknown word for pre-trained word2vec data
        mean.append(wv.vectors_norm[wv.vocab["apple"].index])
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def word_averaging_list(wv, text_list):
  return np.vstack([word_averaging(wv, review) for review in text_list ])

def tokensation(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            tokens.append(word)
    return tokens
  
def x_to_word2vec(df,word2vec_model):
    claim_list = df.apply(lambda r: tokensation(r['claim']), axis=1).values
    sentence_list = df.apply(lambda r: tokensation(r['sentence']), axis=1).values
    claim = word_averaging_list(word2vec_model,claim_list)
    sentence = word_averaging_list(word2vec_model,sentence_list)
    return np.hstack([claim,sentence])


In [0]:
from sklearn.model_selection import train_test_split

word2vec_model_path = '/drive/My Drive/IRDM/word2vec/glove.6B.50d.txt' 
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=False,unicode_errors='ignore',encoding='utf-8')
word2vec_model.init_sims(replace=True)
df = pd.read_csv('/drive/My Drive/IRDM//task4/tas4_train.csv')
df = df.dropna(how='any',axis=0) 
X=x_to_word2vec(df,word2vec_model)
Y=df['label'].values
X_train,X_valid,y_train,y_valid = train_test_split(X,Y)


##Logistic Regression

In [0]:
import numpy as np

class Logistic_Regression:
    def _init__(self):
        self.coef = None
        self.intercept = None
        self.theta = None
        
    def save(self,path):
        np.save(path+'coef.npy',self.coef)
        np.save(path+'intercept.npy',self.intercept)
        np.save(path+'theta.npy',self.theta)
        
    def load(self,path):
        self.coef = np.load(path+'coef.npy')
        self.intercept = np.load(path+'intercept.npy')
        self.theta = np.load(path+'theta.npy')
        
    def sigmod(self, t):
        return 1. / (1. + np.exp(-t))

    def loss(self,theta, X_b, y):
        y_hat = self.sigmod(X_b.dot(theta))
        return -1 * (np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))) / len(y)

    def gradient(self,theta, X_b, y):
        y_hat = self.sigmod(X_b.dot(theta))
        return X_b.T.dot(y_hat - y) / len(y)

    def gradient_descent(self,X_b, y, initial_theta, eta, n_iters, epsilon=1e-6):
        theta = initial_theta
        cur_iter = 0
        while cur_iter < n_iters:
            gradient = self.gradient(theta, X_b, y)
            pre_theta = theta
            theta = theta-eta* gradient
            if(cur_iter%100==0):
              file=open("/drive/My Drive/IRDM/task4/model_loss_"+str(eta)+".txt",'a')
              file.write("iter:"+str(cur_iter)+";"+str(abs(self.loss(theta, X_b, y)))+"\n")
              print("iter:",cur_iter, " : ", abs(self.loss(theta, X_b, y)))
              file.close()
            if abs(self.loss(theta, X_b, y) - self.loss(pre_theta, X_b, y)) < epsilon:
                break
            cur_iter += 1
        return theta
     
    def fit(self, X_train, y_train, eta=1, n_iters=1e6):
        X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
        initial_theta = np.zeros(X_b.shape[1])
        self.theta = self.gradient_descent(X_b, y_train, initial_theta, eta, n_iters)
        self.intercept = self.theta[0]
        self.coef = self.theta[1:]

    def predict_proability(self, X):
        X_b = np.hstack([np.ones((len(X), 1)), X])
        return self.sigmod(X_b.dot(self.theta))

    def get_classification(self, X):
        proability = self.predict_proability(X)
        res=[]
        for prob in proability:
            if prob>=0.5:
                res.append(1)
            else:
                res.append(0)
        return np.array(res, dtype='int')

    def Accuracy(self, X, y):
        y_predict = self.predict_proability(X)
        accu=0
        for i in range(len(y_predict)):
          if(y_predict[i]>=0.5 and y[i]==1):
            accu+=1
          if(y_predict[i]<0.5 and y[i]==0):
            accu+=1
        print('Accurately predicted:' +str(accu))
        print('total: '+str(len(y_predict)))
        return accu/ len(y_predict)
        
    def True_Negative(self, X, y):
        y_predict = self.get_classification(X)
        sum_z_z=0
        for i in range(len(y)):
          if(y[i]==0 and y_predict[i]==0):
            sum_z_z+=1
        return sum_z_z

    def False_Negative(self, X, y):
        y_predict = self.get_classification(X)
        sum_o_z=0
        for i in range(len(y)):
          if(y[i]==1 and y_predict[i]==0):
            sum_o_z+=1
        return sum_o_z

    def True_Positive(self, X, y):
        y_predict = self.get_classification(X)
        sum_o_o=0
        for i in range(len(y)):
          if(y[i]==1 and y_predict[i]==1):
            sum_o_o+=1
        return sum_o_o

    def False_Positive(self, X, y):
        y_predict = self.get_classification(X)
        sum_z_o=0
        for i in range(len(y)):
          if(y[i]==0 and y_predict[i]==1):
            sum_z_o+=1
        return sum_z_o

    def precision(self,X,y):
        tp=self.True_Positive(X,y)
        fp=self.False_Positive(X,y)
        return tp/(tp+fp)

    def recall(self,X,y):
        tp=self.True_Positive(X,y)
        fn=self.False_Negative(X,y)
        return tp/(tp+fn)

    def f1_score(self,X,y):
        return 2*self.precision(X,y)*self.recall(X,y)/(self.precision(X,y)+self.recall(X,y))

##Train model

In [0]:
logistic_regression = Logistic_Regression()
logistic_regression.fit(X_train,y_train,n_iters=100000,eta=1)
logistic_regression.save('/drive/My Drive/IRDM/task4/LR_model/')

In [0]:
logistic_regression= LogisticRegression()
logistic_regression.load('/drive/My Drive/IRDM/task4/LR_model/')

print ("Validation Accuracy:",logistic_regression.accuracy(X_valid,y_valid))

##Test model

In [0]:
from numpy import random

def task4_pre(documents):
  for document in documents:
    print(document)
    train_set=pd.DataFrame()
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document)
    train=open('/drive/My Drive/IRDM/shared_task_dev.jsonl','r')
    length=len(temp_csv)
    
    
    for line in train:
      json_obj=json.loads(line)
      if json_obj['verifiable']!='VERIFIABLE':
        continue
      elif json_obj['id'] in [137334, 111897, 89891, 181634, 219028, 108281, 204361, 54168, 105095, 18708]:
        evidence=json_obj['evidence']
        dict_temp={}
        for item in evidence:
          for sub_item in item:
            if sub_item[2] in dict_temp:
              info[sub_item[2]].append(sub_item[3])
            else:
              info[sub_item[2]]=[]
              info[sub_item[2]].append(sub_item[3])
              
              
              
        for id_1 in dict_temp.keys():
          sentence_line=temp_csv[temp_csv['id']==id_1]
          if len(sentence_line)!=1:
            continue
          else:
            sentence=''
            for line_num in dict_temp[id_1]:
              arr=eval(sentence_line['lines'].values[0])
              if(line_num>=len(arr)):
                continue
              sentence=sentence+arr[line_num]
            r_idx=random.randint(length)
            negative_line=eval(temp_csv.iloc[r_idx]['lines'])
            while len(negative_line)==0:
              r_idx=random.randint(length)
              negative_line=eval(temp_csv.iloc[r_idx]['lines'])
            neg_sentence=negative_line[random.randint(len(negative_line))]
            while neg_sentence==sentence:
              r_idx=random.randint(length)
              negative_line=eval(temp_csv.iloc[r_idx]['lines'])

            neg_sentence=negative_line[random.randint(len(negative_line))]
            train_set=train_set.append({'claim':json_obj['claim'],'sentence':sentence,'label':1},ignore_index=True)
            train_set=train_set.append({'claim':json_obj['claim'],'sentence':neg_sentence,'label':0},ignore_index=True)
    train_set.to_csv('/drive/My Drive/IRDM/task4/task4_testset/'+document[:8]+'_test.csv')


In [0]:
document_all = os.listdir('/drive/My Drive/IRDM/task3/document/')
document_all=sorted(document_all)
bound=[0,len(document_all)]
documents=document_all[bound[0]:bound[1]]
pre_processing_test_t4(documents)

In [0]:
def merge_test_test4()
    output = r'/drive/My Drive/IRDM/task4/task4_test.csv' 
    file_list = os.listdir('/drive/My Drive/IRDM/task4/task4_testset/')
    df = pd.read_csv('/drive/My Drive/IRDM/task4/task4_testset/'+file_list[0])
    df['claim']=None
    df['label']=None
    df['sentence']=None
    df = df.dropna(how='any',axis=0) 
    df.to_csv(output, encoding="utf_8", index=False) 
    for i in range(1, len(file_list)):
        df = pd.read_csv('/drive/My Drive/IRDM/task4/task4_testset/'+file_list[i])
        df = df.dropna(how='any',axis=0) 
        df.to_csv(output, encoding="utf_8", index=False, header=False, mode='a+')
        
merge_test_test4

In [0]:
###Validation
test_set=pd.read_csv('/drive/My Drive/IRDM/task4/task4_test.csv')
X=x_to_word2vec(test_set,word2vec_model)
Y=test_set['label'].values
logistic_regression= Logistic_Regression()
logistic_regression.load('/drive/My Drive/IRDM/task4/LR_model/')
y_prob=logistic_regression.predict_proability(X)
y_predict=logistic_regression.get_classification(X)
claims=test_set['claim'].values
sentence=test_set['sentence'].values
for i in range(len(y_prob)):
  if(i%100==0):
    print('label: ',Y[i],'probability: ',y_prob[i],'classification: ',y_predict[i],'claim: ',claims[i],'sentence: ',sentence[i])
    print("!!!!")
    
print ("Validation Accuracy:",logistic_regression.Accuracy(X,Y))


#5. Task5

##Testset Generation

In [0]:
import pandas as pd
import os
import time
from numpy import random

def task5_testset_generation(documents):
  for document in documents:
    print(document)
    train_set=pd.DataFrame()
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document)
    train=open('/drive/My Drive/IRDM/shared_task_dev.jsonl','r')
    length=len(temp_csv)
    for line in train:
      json_obj=json.loads(line)
      if json_obj['verifiable']!='VERIFIABLE':
        continue
      else:
        evidence=json_obj['evidence']
        info={}
        for item in evidence:
          for sub_item in item:
            if sub_item[2] in info:
              info[sub_item[2]].append(sub_item[3])
            else:
              info[sub_item[2]]=[]
              info[sub_item[2]].append(sub_item[3])
        for id in info.keys():
          sentence_line=temp_csv[temp_csv['id']==id]
          if len(sentence_line)!=1:
            continue
          else:
            sentence=''
            for line_num in info[id]:
              arr=eval(sentence_line['lines'].values[0])
              if(line_num>=len(arr)):
                continue
              sentence=sentence+arr[line_num]
            r_idx=random.randint(length)
            negative_line=eval(temp_csv.iloc[r_idx]['lines'])
            while len(negative_line)==0:
              r_idx=random.randint(length)
              negative_line=eval(temp_csv.iloc[r_idx]['lines'])
            neg_sentence=negative_line[random.randint(len(negative_line))]
            while neg_sentence==sentence:
              r_idx=random.randint(length)
              negative_line=eval(temp_csv.iloc[r_idx]['lines'])

            neg_sentence=negative_line[random.randint(len(negative_line))]
            train_set=train_set.append({'claim':json_obj['claim'],'sentence':sentence,'label':1},ignore_index=True)
            train_set=train_set.append({'claim':json_obj['claim'],'sentence':neg_sentence,'label':0},ignore_index=True)
    train_set.to_csv('/drive/My Drive/IRDM/task5/testset/'+document[:8]+'_test.csv')

    
def generate_train_set_t5(bound1,bound2)
    docu_all = os.listdir('/drive/My Drive/IRDM/task3/document/')
    docu_all=sorted(docu_all)
    documents=docu_all[bound1:bound2]
    task5_testset_generation(documents)

generate_train_set_t5(1,109)


In [0]:
def merge_file2(inputpath,outputfile)
    output = r'/drive/My Drive/IRDM/'+outputfile
    file_list = os.listdir('/drive/My Drive/IRDM/'+inputpath)
    df = pd.read_csv('/drive/My Drive/IRDM/task5/testset/'+file_list[0])
    df['claim']=None
    df['label']=None
    df['sentence']=None
    df = df.dropna(how='any',axis=0) 
    df.to_csv(output, encoding="utf_8", index=False)
    for i in range(1, len(file_list)):
        print(i)
        df = pd.read_csv('/drive/My Drive/IRDM/'+inputpath'/'+file_list[i])
        df = df.dropna(how='any',axis=0) 
        df.to_csv(output, encoding="utf_8", index=False, header=False, mode='a+')

In [0]:
merge_file2('task5/testset','task5/test_task5.csv')

##Test Model

In [0]:
def word_averaging(wv, words):
    mean = []
    contain_wv_word = False
    for word in words:
        if word in wv.vocab:
            contain_wv_word = True
            mean.append(wv.vectors_norm[wv.vocab[word].index])
    if not contain_wv_word: #unknown word for pre-trained word2vec data
        mean.append(wv.vectors_norm[wv.vocab["apple"].index])
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def word_averaging_list(wv, text_list):
  return np.vstack([word_averaging(wv, review) for review in text_list ])

def tokensation(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            tokens.append(word)
    return tokens
  
def x_to_word2vec(df,word2vec_model):
    claim_list = df.apply(lambda r: tokensation(r['claim']), axis=1).values
    sentence_list = df.apply(lambda r: tokensation(r['sentence']), axis=1).values
    claim = word_averaging_list(word2vec_model,claim_list)
    sentence = word_averaging_list(word2vec_model,sentence_list)
    return np.hstack([claim,sentence])

Computer Precision Recall F1 in Task4's class Logistic Regression

In [0]:
test=pd.read_csv('/drive/My Drive/IRDM/task5/test_task5.csv')

X=x_to_word2vec(test,word2vec_model)
Y=test['label'].values
logistic_regression= LogisticRegression()
logistic_regression.load('/drive/My Drive/IRDM/task4/LR_model/')
y_prob=logistic_regression.predict_proability(X)
y_predict=logistic_regression.get_classification(X)
print ("Test Accuracy:",logistic_regression.accuracy(X,Y)
print("Precision :",logistic_regression.precision(X,Y))
print("Recall:",logistic_regression.recall(X,Y))
print("f1:",logistic_regression.f1_score(X,Y))

#Task6

##Data preprocessing

In [0]:
import time
from numpy import random
import pandas as pd

def test6_trainset_generation(documents):
  
  
  for document in documents:
    print(document)
    train_set=pd.DataFrame()
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document)
    train=open('/drive/My Drive/IRDM/train.jsonl','r')
    length=len(temp_csv)
    for line in train:
      json_obj=json.loads(line)
      if json_obj['verifiable']!='VERIFIABLE':
        continue
      else:
        evidence=json_obj['evidence']
        info={}
        for item in evidence:
          for sub_item in item:
            if sub_item[2] in info:
              info[sub_item[2]].append(sub_item[3])
            else:
              info[sub_item[2]]=[]
              info[sub_item[2]].append(sub_item[3])
        for id in info.keys():
          sentence_line=temp_csv[temp_csv['id']==id]
          if len(sentence_line)!=1:
            continue
          else:
            sentence=''
            for line_num in info[id]:
              arr=eval(sentence_line['lines'].values[0])
              if(line_num>=len(arr)):
                continue
              sentence=sentence+arr[line_num]
            
            
            label=0
            if(json_obj['label']=='SUPPORTS'):
              label=1
            train_set=train_set.append({'claim':json_obj['claim'],'sentence':sentence,'label':label},ignore_index=True)

    train_set.to_csv('/drive/My Drive/IRDM/task6/train_set/'+document[:8]+'_train.csv')

    
def generate_train_set_t6(bound1,bound2)
    docu_all = os.listdir('/drive/My Drive/IRDM/task3/document/')
    docu_all=sorted(docu_all)
    documents=docu_all[bound1:bound2]
    task5_testset_generation(documents)

test6_trainset_generation(1,109)


In [0]:
merge_file2('task6a/train_task6.csv','task6/train_set/')

In [0]:
import pandas as pd
from numpy import random

def pre_processing_t6_test(documents):
  for document in documents:
    print(document)
    train_set=pd.DataFrame()
    temp_csv=pd.read_csv('/drive/My Drive/IRDM/task3/document/'+document)
    train=open('/drive/My Drive/IRDM/shared_task_dev.jsonl','r')
    length=len(temp_csv)
    for line in train:
      json_obj=json.loads(line)
      if json_obj['verifiable']!='VERIFIABLE':
        continue
      else:
        evidence=json_obj['evidence']
        info={}
        for item in evidence:
          for sub_item in item:
            if sub_item[2] in info:
              info[sub_item[2]].append(sub_item[3])
            else:
              info[sub_item[2]]=[]
              info[sub_item[2]].append(sub_item[3])
        for id in info.keys():
          sentence_line=temp_csv[temp_csv['id']==id]
          if len(sentence_line)!=1:
            continue
          else:
            sentence=''
            for line_num in info[id]:
              arr=eval(sentence_line['lines'].values[0])
              if(line_num>=len(arr)):
                continue
              sentence=sentence+arr[line_num]
            sup=0
            if(json_obj['label']=='SUPPORTS'):
              sup=1
            train_set=train_set.append({'claim':json_obj['claim'],'sentence':sentence,'label':sup},ignore_index=True)

    train_set.to_csv('/drive/My Drive/IRDM/task6/test_set/'+document[:8]+'_test.csv')

documents_traverse = os.listdir('/drive/My Drive/IRDM/task3/document/')
documents_traverse=sorted(documents_traverse)
bound=[0,len(documents_traverse)]
documents=documents_traverse[bound[0]:bound[1]]
pre_processing_t6_test(documents)


In [0]:
merge_file2('task6/test_task6.csv','task6/test_set/')

##Model

In [0]:
import keras
from keras import Model
from keras.layers import *
from keras.preprocessing import text,sequence

class NN_Model():
    def model(self, embeddings_matrix, maxlen_cl,maxlen_sen, word_index):
        claim = Input(shape=(maxlen_cl,),name='claim')
        sentence=Input(shape=(maxlen_sen,),name='sentence')
        encode_c = CuDNNGRU(64, return_sequences=True)
        encode_s= Bidirectional(CuDNNGRU(64, return_sequences=True))
        encode2_s = Bidirectional(CuDNNGRU(64, return_sequences=True))
        x_4_s = Embedding(len(word_index) + 1,
                        embeddings_matrix.shape[1],
                        weights=[embeddings_matrix],
                        input_length=maxlen_sen,
                        trainable=True)(sentence)
        x_4_c = Embedding(len(word_index) + 1,
                embeddings_matrix.shape[1],
                weights=[embeddings_matrix],
                input_length=maxlen_cl,
                trainable=True)(claim)
        x_3_s = SpatialDropout1D(0.2)(x_4_s)
        x_3_s = encode_s(x_3_s)
        x_3_s = Dropout(0.2)(x_3_s)
        x_3_s = encode2_s(x_3_s)
        x_3_s = Dropout(0.2)(x_3_s)
        avg_pool_3_s = GlobalAveragePooling1D()(x_3_s)
        max_pool_3_s = GlobalMaxPooling1D()(x_3_s)
        
        x_3_c = SpatialDropout1D(0.2)(x_4_c)
        x_3_c = encode_c(x_3_c)
        x_3_c = Dropout(0.2)(x_3_c)
        avg_pool_3_c = GlobalAveragePooling1D()(x_3_c)
        max_pool_3_c = GlobalMaxPooling1D()(x_3_c)
        x = keras.layers.concatenate([avg_pool_3_s, max_pool_3_s,avg_pool_3_c, max_pool_3_c], name="fc")
        main_output = Dense(1, activation="sigmoid",name='main_output')(x)
        
        adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08,amsgrad=True)
        rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
        model = Model(inputs=[claim,sentence], outputs=main_output)
        model.compile(
            loss='binary_crossentropy',
            metrics = ['accuracy'],
            optimizer=adam)
        return model
      
from keras.callbacks import ModelCheckpoint, Callback
from sklearn.metrics import f1_score, recall_score, precision_score

def getClassification(prob):
   
    if prob<0.5:
        return 0
    else :
        return 1

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        print('epoch end')
        print(len(self.validation_data))
        val_predict = list(map(getClassification, self.model.predict([self.validation_data[0],self.validation_data[1]])))
        val_targ = list(self.validation_data[2])
        _val_f1 = f1_score(val_targ, val_predict, average="macro")
        _val_recall = recall_score(val_targ, val_predict, average="macro")
        _val_precision = precision_score(val_targ, val_predict, average="macro")
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(_val_f1, _val_precision, _val_recall)
        print("max f1")
        print(max(self.val_f1s))
        return

##Train and validation

In [0]:
from sklearn.model_selection import train_test_split
from keras.utils import plot_model
import pandas as pd

data=pd.read_csv('/drive/My Drive/IRDM/task6/train_task6.csv')
model_dir='/drive/My Drive/IRDM/task6_model/'
data['sentence'] = data.apply(lambda r: r['sentence'].replace('\t',''), axis=1)
x_train_c=data['claim'].values
x_train_s=data['sentence'].values
y_train=data['label'].values

word2vec_model_path = '/drive/My Drive/IRDM/word2vec/glove.6B.50d.txt' 
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=False,unicode_errors='ignore',encoding='utf8')
word2vec_model.init_sims(replace=True)

tokenizer = text.Tokenizer(num_words=None)
data_input=np.concatenate((data['claim'].values,data['sentence'].values),axis=0)
tokenizer.fit_on_texts(data_input)
word_index = tokenizer.word_index
embeddings_matrix = np.zeros((len(word_index) + 1, word2vec_model.vector_size))
vocab_list = [(k, word2vec_model[k]) for k, v in word2vec_model.vocab.items()]
for word, i in word_index.items():
    if word in word2vec_model:
        embedding_vector = word2vec_model[word]
        #print(embedding_vector)
    else:
        embedding_vector = None
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector


In [0]:
maxlen_s=500
maxlen_c=50

model = NN_Model().model(embeddings_matrix, maxlen_c,maxlen_s, word_index)

file_path = model_dir + "model_{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file_path, verbose=1, save_weights_only=True)
metrics = Metrics()
callbacks_list = [checkpoint, metrics]
x_train_c = tokenizer.texts_to_sequences(x_train_c)
x_train_c = sequence.pad_sequences(x_train_c, maxlen=maxlen_c)
x_train_s = tokenizer.texts_to_sequences(x_train_s)
x_train_s = sequence.pad_sequences(x_train_s, maxlen=maxlen_s)

In [0]:
history = model.fit(x={'claim':x_train_c,'sentence':x_train_s},y={'main_output':y_train},validation_split=0.2, epochs=15,callbacks=callbacks_list, verbose=1,batch_size=512)


In [0]:
data=pd.read_csv('/drive/My Drive/IRDM/task6/test_task6.csv')

data['sentence'] = data.apply(lambda r: sentence_clean(r['sentence']), axis=1)
x_test_c=data['claim'].values
x_test_s=data['sentence'].values
y_test=data['label'].values


maxlen_s=500
maxlen_c=50

callbacks_list = [checkpoint, metrics]
x_test_c = tokenizer.texts_to_sequences(x_test_c)
x_test_c = sequence.pad_sequences(x_test_c, maxlen=maxlen_c)
x_test_s = tokenizer.texts_to_sequences(x_test_s)
x_test_s = sequence.pad_sequences(x_test_s, maxlen=maxlen_s)



In [0]:
from keras.models import load_model
best=NN_Model().model(embeddings_matrix, maxlen_c,maxlen_s, word_index)
best.load_weights('./drive/My Drive/my_irdm/task5_model/model_04.hdf5')
print(model_best.metrics_names)

final = model_best.evaluate(x={'claim':x_test_c,'sentence':x_test_s},
            y={'main_output': y_test},
            batch_size=10, verbose=1)
print(final)

In [0]:
plot_model(model, to_file='/drive/My Drive/IRDM/model.png')

#Task 8

##Attention

In [0]:
# coding=utf8
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.layers.merge import _Merge


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        input_shape = K.int_shape(x)

        features_dim = self.features_dim
        # step_dim = self.step_dim
        step_dim = input_shape[1]

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b[:input_shape[1]]

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    	# print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        # return input_shape[0], input_shape[-1]
    	return input_shape[0], self.features_dim
# end Attention


class JoinAttention(_Merge):
    def __init__(self, step_dim, hid_size,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism according to other vector.
        Supports Masking.
        # Input shape, list of
            2D tensor with shape: `(samples, features_1)`.
            3D tensor with shape: `(samples, steps, features_2)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            en = LSTM(64, return_sequences=False)(input)
            de = LSTM(64, return_sequences=True)(input2)
            output = JoinAttention(64, 20)([en, de])
        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.hid_size = hid_size
        super(JoinAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        if not isinstance(input_shape, list):
            raise ValueError('A merge layer [JoinAttention] should be called '
                             'on a list of inputs.')
        if len(input_shape) != 2:
            raise ValueError('A merge layer [JoinAttention] should be called '
                             'on a list of 2 inputs. '
                             'Got ' + str(len(input_shape)) + ' inputs.')
        if len(input_shape[0]) != 2 or len(input_shape[1]) != 3:
            raise ValueError('A merge layer [JoinAttention] should be called '
                             'on a list of 2 inputs with first ndim 2 and second one ndim 3. '
                             'Got ' + str(len(input_shape)) + ' inputs.')

        self.W_en1 = self.add_weight((input_shape[0][-1], self.hid_size),
                                 initializer=self.init,
                                 name='{}_W0'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.W_en2 = self.add_weight((input_shape[1][-1], self.hid_size),
                                 initializer=self.init,
                                 name='{}_W1'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.W_de = self.add_weight((self.hid_size,),
                                 initializer=self.init,
                                 name='{}_W2'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)

        if self.bias:
            self.b_en1 = self.add_weight((self.hid_size,),
                                     initializer='zero',
                                     name='{}_b0'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.b_en2 = self.add_weight((self.hid_size,),
                                     initializer='zero',
                                     name='{}_b1'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.b_de = self.add_weight((input_shape[1][1],),
                                     initializer='zero',
                                     name='{}_b2'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b_en1 = None
            self.b_en2 = None
            self.b_de = None

        self._reshape_required = False
        self.built = True

    def compute_output_shape(self, input_shape):
        return input_shape[1][0], input_shape[1][-1]

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, inputs, mask=None):
        en = inputs[0]
        de = inputs[1]
        de_shape = K.int_shape(de)
        step_dim = de_shape[1]

        hid_en = K.dot(en, self.W_en1)
        hid_de = K.dot(de, self.W_en2)
        if self.bias:
            hid_en += self.b_en1
            hid_de += self.b_en2
        hid = K.tanh(K.expand_dims(hid_en, axis=1) + hid_de)
        eij = K.reshape(K.dot(hid, K.reshape(self.W_de, (self.hid_size, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b_de[:step_dim]

        a = K.exp(eij - K.max(eij, axis=-1, keepdims=True))

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask[1], K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = de * a
        return K.sum(weighted_input, axis=1)

##Model

In [0]:
import keras
from keras import Model
from keras.layers import *
from keras.preprocessing import text,sequence

class Model_new():
    def model(self, embeddings_matrix, maxlen_cl,maxlen_sen, word_index):
        claim = Input(shape=(maxlen_cl,),name='claim')
        sentence=Input(shape=(maxlen_sen,),name='sentence')
        attention_s=Attention(maxlen_sen)
        attention_c=Attention(maxlen_cl)
        encode_c = CuDNNGRU(64, return_sequences=True)
        encode_s= Bidirectional(CuDNNGRU(64, return_sequences=True))
        encode2_s = Bidirectional(CuDNNGRU(64, return_sequences=True))
        x_4_s = Embedding(len(word_index) + 1,
                        embeddings_matrix.shape[1],
                        weights=[embeddings_matrix],
                        input_length=maxlen_sen,
                        trainable=True)(sentence)
        x_4_c = Embedding(len(word_index) + 1,
                embeddings_matrix.shape[1],
                weights=[embeddings_matrix],
                input_length=maxlen_cl,
                trainable=True)(claim)
        x_3_s = SpatialDropout1D(0.2)(x_4_s)
        x_3_s = encode_s(x_3_s)
        x_3_s = Dropout(0.2)(x_3_s)
        x_3_s = encode2_s(x_3_s)
        x_3_s = Dropout(0.2)(x_3_s)
        avg_pool_3_s = GlobalAveragePooling1D()(x_3_s)
        max_pool_3_s = GlobalMaxPooling1D()(x_3_s)
        attention_3_s=attention_s(x_3_s)
        
        x_3_c = SpatialDropout1D(0.2)(x_4_c)
        x_3_c = encode_c(x_3_c)
        x_3_c = Dropout(0.2)(x_3_c)
        avg_pool_3_c = GlobalAveragePooling1D()(x_3_c)
        max_pool_3_c = GlobalMaxPooling1D()(x_3_c)
        attention_3_c=attention_c(x_3_c)
        
        x = keras.layers.concatenate([avg_pool_3_s, max_pool_3_s,attention_3_s,avg_pool_3_c, max_pool_3_c,attention_3_c], name="fc")
        main_output = Dense(1, activation="sigmoid",name='main_output')(x)
        
        adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08,amsgrad=True)
        rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
        model = Model(inputs=[claim,sentence], outputs=main_output)
        model.compile(
            loss='binary_crossentropy',
            metrics = ['accuracy'],
            optimizer=adam)
        return model
      
from keras.callbacks import ModelCheckpoint, Callback
from sklearn.metrics import f1_score, recall_score, precision_score

def getClassification(prob):
    if prob<0.5:
        return 0
    else :
        return 1

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        print('epoch end')
        print(len(self.validation_data))
        val_predict = list(map(getClassification, self.model.predict([self.validation_data[0],self.validation_data[1]])))
        val_targ = list(self.validation_data[2])
        _val_f1 = f1_score(val_targ, val_predict, average="macro")
        _val_recall = recall_score(val_targ, val_predict, average="macro")
        _val_precision = precision_score(val_targ, val_predict, average="macro")
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(_val_f1, _val_precision, _val_recall)
        print("max f1")
        print(max(self.val_f1s))
        return

##Train and Validation

In [0]:
from sklearn.model_selection import train_test_split
from keras.utils import plot_model
import pandas as pd
def sentence_clean(text):
  text=text.replace('\t','')
  return text

data=pd.read_csv('/drive/My Drive/IRDM/task6/train_task6.csv')
model_dir='/drive/My Drive/IRDM/task6_model/'
data['sentence'] = data.apply(lambda r: sentence_clean(r['sentence']), axis=1)
x_train_c=data['claim'].values
x_train_s=data['sentence'].values
y_train=data['label'].values

word2vec_model_path = '/drive/My Drive/IRDM/word2vec/glove.6B.50d.txt' 
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=False,unicode_errors='ignore',encoding='utf8')
word2vec_model.init_sims(replace=True)

tokenizer = text.Tokenizer(num_words=None)
to_fit=np.concatenate((data['claim'].values,data['sentence'].values),axis=0)

tokenizer.fit_on_texts(to_fit)
word_index = tokenizer.word_index
embeddings_matrix = np.zeros((len(word_index) + 1, word2vec_model.vector_size))
vocab_list = [(k, word2vec_model[k]) for k, v in word2vec_model.vocab.items()]
for word, i in word_index.items():
    if word in word2vec_model:
        embedding_vector = word2vec_model[word]
        #print(embedding_vector)
    else:
        embedding_vector = None
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector


In [0]:
maxlen_s=500
maxlen_c=50
model = Model_new().model(embeddings_matrix, maxlen_c,maxlen_s, word_index)

file_path = model_dir + "model_new_{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file_path, verbose=1, save_weights_only=True)
metrics = Metrics()
callbacks_list = [checkpoint, metrics]
x_train_c = tokenizer.texts_to_sequences(x_train_c)
x_train_c = sequence.pad_sequences(x_train_c, maxlen=maxlen_c)
x_train_s = tokenizer.texts_to_sequences(x_train_s)
x_train_s = sequence.pad_sequences(x_train_s, maxlen=maxlen_s)


In [0]:
history = model.fit(x={'claim':x_train_c,'sentence':x_train_s},y={'main_output':y_train},validation_split=0.2, epochs=15,callbacks=callbacks_list, verbose=1,batch_size=512)


In [0]:
data=pd.read_csv('/drive/My Drive/IRDM/task6/test_task6.csv')

data['sentence'] = data.apply(lambda r: sentence_clean(r['sentence']), axis=1)
x_test_c=data['claim'].values
x_test_s=data['sentence'].values
y_test=data['label'].values


maxlen_s=500
maxlen_c=50

callbacks_list = [checkpoint, metrics]
x_test_c = tokenizer.texts_to_sequences(x_test_c)
x_test_c = sequence.pad_sequences(x_test_c, maxlen=maxlen_c)
x_test_s = tokenizer.texts_to_sequences(x_test_s)
x_test_s = sequence.pad_sequences(x_test_s, maxlen=maxlen_s)

In [0]:
from keras.models import load_model
best=Model_new().model(embeddings_matrix, maxlen_c,maxlen_s, word_index)
best.load_weights('/drive/My Drive/IRDM/task8/model/model_new_09.hdf5')
print(model_best.metrics_names)

final = model_best.evaluate(x={'claim':x_test_c,'sentence':x_test_s},
            y={'main_output': y_test},
            batch_size=10, verbose=1)
print(final)

In [0]:
plot_model(model, to_file='/drive/My Drive/IRDM/model_task8.png')